# Model 3: Predict Star Formation Variables (sSFR, SFR, M*, age) Based on Visual Morphology (galaxy image)


In [2]:
#Loading needed modules and classes/functions 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import marvin
from marvin.tools.maps import Maps
from marvin.tools.image import Image
from marvin.utils.general.images import get_images_by_list
from marvin import config
from marvin.tools.cube import Cube
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import os
import shutil

#set config attributes and turn on global downloads of Marvin data
config.setRelease('DR15')
config.mode = 'local'
config.download = True

#3 Linear layers NN, 1 hidden 
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
        self.linear1 = torch.nn.Linear(outputSize, outputSize)
        self.ReLU= torch.nn.ReLU()

    def forward(self, x):
        x = self.linear(x)
        x = self.ReLU(x)
        x = self.linear1(x)
        return x


[INFO]: No release version set. Setting default to DR15
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


# Importing Data from Schema Table

In [41]:


data=pd.read_csv('CompleteTable.csv')  #Importing All MaNGA Data from DPRall Schema

galaxy_list=np.loadtxt('Query Results',dtype=str) #Pulling Manga ID's of galaxies which satisfy log(M) > 9 and 0 < z < 0.1

galaxy_index=np.zeros(len(galaxy_list)) 
for i in range (len(galaxy_list)): #Getting the index of these galaxies in the schema table
    galaxy_index[i]=np.where(data.loc[:,'mangaid']==galaxy_list[i])[0][0]

galaxy_index=np.array(galaxy_index,dtype=int) #Ensuring we have array that can be used to index, force int 

galaxies=data.iloc[galaxy_index] #DF of galaxies which satisfies the condition, contains all relevant schema data 

#Problem with image associated with manga id at galaxy_list[3548], mangaid- 1-135668
galaxy_list=np.delete(galaxy_list,3548)

#Creating the arrays of the independent variables were are interested in, and dependent variable n 

mass=galaxies.loc[:,'nsa_sersic_mass']
log_mass=np.log10(mass)

SFR=galaxies.loc[:,'sfr_tot']
log_SFR=np.log10(SFR)

ha_flux=galaxies.loc[:,'emline_gflux_tot_ha_6564']

n=galaxies.loc[:,'nsa_sersic_n']
n=np.array(n,dtype=np.float32)
n=torch.from_numpy(n).to('cuda:0').reshape(-1,1)


# Importing Images from their Downloaded Locations 


In [67]:
# The images are in a folder named 'input/natural-images/natural_images'
images=Image.from_list(galaxy_list[5:15])
print(images[0])    

<Marvin Image (plateifu='8553-6102', mode='local', data-origin='file')>


In [67]:
image_locations=[]
for i in range (len(galaxy_list)):
    image_locations.append(Image(galaxy_list[i]).filename)
    

[WARNING]: more than one plate-ifu found for mangaid=1-210186. Using the one with the highest SN2. (MarvinUserWarning)
[WARNING]: more than one plate-ifu found for mangaid=1-209770. Using the one with the highest SN2. (MarvinUserWarning)
[WARNING]: more than one plate-ifu found for mangaid=1-209772. Using the one with the highest SN2. (MarvinUserWarning)
[WARNING]: more than one plate-ifu found for mangaid=1-209786. Using the one with the highest SN2. (MarvinUserWarning)
[WARNING]: more than one plate-ifu found for mangaid=1-209823. Using the one with the highest SN2. (MarvinUserWarning)
[WARNING]: more than one plate-ifu found for mangaid=1-235398. Using the one with the highest SN2. (MarvinUserWarning)
[WARNING]: more than one plate-ifu found for mangaid=1-235530. Using the one with the highest SN2. (MarvinUserWarning)
[WARNING]: more than one plate-ifu found for mangaid=1-592881. Using the one with the highest SN2. (MarvinUserWarning)
[WARNING]: more than one plate-ifu found for man

/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3/9035/stack/images/1901.png


In [48]:


training_folder_name = r"/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3"

img_size=(562,562)

Plates=sorted(os.listdir(training_folder_name))
print(Plates)

TypeError: Mismatch between array dtype ('<U1') and format specifier ('%.18e')